# Company Events Calendar

This Notebook provides an example of retrieving company events for a list of stocks filtered on events types and a range of dates using Refinitiv Data Libraries.

In [1]:
import ipywidgets as widgets
import pandas as pd
import datetime as dt
import refinitiv.data as rd

rd.open_session()

<refinitiv.data.session.Definition object at 0x137c6f640 {name='workspace'}>

In [2]:
event_types_map = {'Brokerage analyst calls': 'BACALL',
                   'Brokerage analyst meetings': 'BAM',
                   'Company visits': 'CV',
                   'Conference presentations': 'CONFP',
                   'Conferences': 'CONF',
                   'Corporate analyst meetings': 'CAM',
                   'Corporate calls and presentations': 'CCALL',
                   'Earnings calls and presentations': 'ECALL',
                   'Earnings releases': 'RES',
                   'Ex Dividends': 'EXDIV',
                   'General & political events': 'GPE',
                   'Guidance calls and presentations': 'GCALL',
                   'IPO filings': 'IPOF',
                   'IPO lockup expiration': 'IPOLE',
                   'IPO pricings': 'IPOP',
                   'IPO withdrawals': 'IPOW',
                   'Market holidays': 'MH',
                   'Merger & Acquisition calls and presentations': 'MACALL',
                   'Other brokerages': 'OTHB',
                   'Other Corporate events': 'OTHC',
                   'Sales & Trading statement calls and presentations': 'STCALL',
                   'Sales & Trading statement releases': 'STR',
                   'Secondary filings': 'SECF',
                   'Secondary pricings': 'SECP',
                   'Secondary withdrawals': 'SECW',
                   'Shareholder / Annual meetings': 'SHM',
                   'Stock splits': 'SSP',
                   'Syndicate roadshows': 'SYNR'}
event_types_w = widgets.SelectMultiple(options=event_types_map,
                                       rows=len(event_types_map),
                                       layout=widgets.Layout(width='50%'))
print('Select event types')
print('Multiple values can be selected with shift and/or ctrl (or command) pressed and mouse clicks or arrow keys.')
print('No selection means retrieve all available event types')
display(event_types_w)

Select event types
Multiple values can be selected with shift and/or ctrl (or command) pressed and mouse clicks or arrow keys.
No selection means retrieve all available event types


SelectMultiple(layout=Layout(width='50%'), options={'Brokerage analyst calls': 'BACALL', 'Brokerage analyst me…

In [3]:
event_types = event_types_w.value
if len(event_types) == 0:
    event_types_str = 'ALL'
else:
    event_types_str = event_types[0]
    for i in range(1, len(event_types)):
        event_types_str = event_types_str + ';' + event_types[i]
print('Select the date range for company events')
options_range = ['-10Y', '-5Y', '-3Y', '-1Y', '-6M', '-3M',
                 '-1M', '-1W', '-1D', '0D', '1D', '1W', '2W', '1M', '3M', '1Y']
date_range_w = widgets.SelectionRangeSlider(
    options=[' ' + x + ' ' for x in options_range], index=(9, 13))
display(date_range_w)

Select the date range for company events


SelectionRangeSlider(index=(9, 13), options=(' -10Y ', ' -5Y ', ' -3Y ', ' -1Y ', ' -6M ', ' -3M ', ' -1M ', '…

In [4]:
start_date = options_range[date_range_w.index[0]]
end_date = options_range[date_range_w.index[1]]
instr = rd.get_data('0#.DJI', 'TR.IndexConstituentRIC')
instr = instr['Constituent RIC'].tolist()
# Remove duplicates, None and empty strings from the list
instr = list(dict.fromkeys(instr))
instr = list(filter(None, instr))
instr = list(filter(str.strip, instr))

events_df = rd.get_data(instr, ['TR.EventStartDate', 'TR.EventType', 'TR.EventTitle'],
                        {'SDate': start_date, 'EDate': end_date, 'EventType': event_types_str})
events_df.rename(columns={'Event Start Date': 'Event Date'}, inplace=True)
events_df = events_df[['Event Date', 'Instrument',
                       'Company Event Type', 'Event Title']]
events_df['Event Date'] = pd.to_datetime(events_df['Event Date'])
events_df.dropna(subset=['Event Date'], inplace=True)
events_df

,Event Date,Instrument,Company Event Type,Event Title
0,2023-03-01,GS.N,Conferences,Goldman Sachs Bus Tour
1,2023-03-06,GS.N,Conferences,Goldman Sachs Latin America Conference
2,2023-03-07,GS.N,ConferencePresentations,Goldman Sachs Group Inc at RBC Capital Markets...
3,2023-03-09,GS.N,Conferences,Goldman Sachs European Chemicals and Consumer ...
4,2023-03-15,GS.N,SyndicateRoadshows,Nkt A/S at Goldman Sachs Group Inc Roadshow - ...
...,...,...,...,...
468,2023-03-30,TRV.N,ExDividends,TRVVd.BA Interim Cash Dividend of gross USD 0....
469,2023-03-30,TRV.N,ExDividends,TRVVh.BA Interim Cash Dividend of gross USD 0....
470,2023-03-30,TRV.N,ExDividends,TRVVm.BA Interim Cash Dividend of gross USD 0....
471,2023-03-30,TRV.N,ExDividends,TRVVdm.BA Interim Cash Dividend of gross USD 0...


In [5]:
rd.close_session()